In [1]:
import requests
from loguru import logger
from pathlib import Path
import os
import uuid
from typing import Generator
import numpy as np
import simplejson as json
import torch
from justatom.tooling.dataset import DatasetRecordAdapter
from justatom.etc.schema import Document
from more_itertools import chunked
import json_repair
import polars as pl

from justatom.storing.weaviate import Finder as WeaviateApi

from tqdm import tqdm

### ✔️ ANN document store backed by <a href="https://github.com/weaviate/weaviate">Weaviate</a>

 > First, make sure Docker is up and running. From the project root, run:
```bash
docker-compose up -d
```

❗️ By default, Weaviate runs on port `2211`.

In [2]:
collection_name = "JUSTATOM_COLLECTION_v2"
weaviate_host, weaviate_port, weaviate_grpc_port = "localhost", 2211, 50051

In [3]:
store = await WeaviateApi.find(collection_name, WEAVIATE_HOST=weaviate_host, WEAVIATE_PORT=weaviate_port, WEAVIATE_GRPC_PORT=weaviate_grpc_port)

2026-02-16 06:51:33.522 | INFO     | justatom.storing.weaviate:find:799 - FINDER | collection_name=[JUSTATOM_COLLECTION_v2]
2026-02-16 06:51:33.524 | INFO     | justatom.storing.weaviate:connect:223 - FINDER | collection_schema_name=[JUSTATOM_COLLECTION_v2]
2026-02-16 06:51:33.524 | INFO     | justatom.storing.weaviate:connect:223 - FINDER | collection_schema_name=[JUSTATOM_COLLECTION_v2]
/home/justatom/miniconda3/envs/jarvis/lib/python3.12/site-packages/weaviate/warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client version 4.11.3. The latest version is 4.19.2.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [4]:
n_docs = await store.count_documents()

In [5]:
logger.info(f"For the collection=[{collection_name}] you have N=[{n_docs}] documents")

2026-02-16 06:51:36.107 | INFO     | __main__:<module>:1 - For the collection=[JUSTATOM_COLLECTION_v2] you have N=[1184] documents


In [6]:
all_collections = await store._client.collections.list_all(simple=True)

In [7]:
logger.info(f"COLLECTION | [{', '.join(all_collections)}]")

2026-02-16 06:51:37.463 | INFO     | __main__:<module>:1 - COLLECTION | [Evalvanillalarge, Justatom_collection_v2, Polaroidsaibase, Justatom_collection]


### ✔️ Prepare dataset

 > For this tutorial we use the built-in dataset `polaroids.ai` — snippets from books, movies, and games.

 > ✅ For large datasets, prefer streaming (`lazy=True`) and avoid materializing the whole file in memory.

In [16]:
dataset_name_or_path = Path(os.getcwd()) / ".data" / "polaroids.ai.data.json"

In [17]:
pl_docs = pl.read_json(dataset_name_or_path)

In [18]:
present_columns = pl_docs.columns

In [19]:
present_columns

['title',
 'author',
 'type',
 'has_image',
 'img_path',
 'speaker',
 'keywords_or_phrases',
 'chunk_id',
 'content',
 'queries',
 'answers',
 'are_contexts_present',
 'dialogue_en',
 'dialogue_description',
 'dialogue_speakers_en']

> ❗️`content` is required. `id` is optional and auto-generated if missing.

 > In this tutorial we also map `chunk_id` to `id` and keep metadata lean for stable Weaviate writes.

> ❗️Filter out rows with `null` in required fields before indexing; otherwise the pipeline may fail at write time.

In [21]:
content_col = "content"
chunk_id_col = "chunk_id"
title_col = "title"
labels_col = "queries"

In [22]:
pl_docs = pl_docs.filter((pl.col(content_col).is_not_null()) & (pl.col(chunk_id_col).is_not_null()))

In [23]:
logger.info(f"There are D=[{pl_docs.shape[0]}] unique documents")

2026-02-16 06:55:20.690 | INFO     | __main__:<module>:1 - There are D=[4992] unique documents


> ❗️We keep `keywords_or_phrases`, relevant `queries`, and original `chunk_id` per chunk.

 > ✅ For robust indexing, use `preserve_all_fields=False` so noisy source fields are not blindly propagated to `meta`.

<small>

|  queries (list[str])  |     content: str     |   chunk_id: str   |
|:---------------------:|:--------------------:|:-----------------:|
| 1. ...thinking about 'The Hunger Games' mechanics, if you were in the same shoes as Gale, entering your name forty-two times to feed your fam, how would you strategize your game in the actual Arena? Would you team up or go solo based on these high stakes? <br><br>2.In the universe of 'The Hunger Games', what are tesserae and what do they offer to the participants in the Harvest?    | And here's where the real interest begins. Suppose you're poor and starving. Then you can ask to be included in the Harvest more times than you're entitled to, and in return you'd get tesserae. They give you grain and oil for a whole year for one tessera per person. You won't be full, but it's better than nothing. You can take tesserae for the whole family. When I was twelve, my name was entered four times. Once by law, and once more for tesserae for Prim, my mother, and myself. The next years had to do the same. And since the price of a tessera increases by one entry each year, now that I've turned sixteen, my name will be on twenty cards. Gale is eighteen, and he's been feeding a family of five for seven years. His name will be entered forty two times! It's clear that people like Madge, who has never had to risk because of tesserae, annoy Gale. Next to us, the inhabitants of the slag heap, she simply has no chance of getting into the games. Well, almost no chance. Of course, the rules are set by the Capitol, not the districts, let alone Madge's relatives, and it's still hard to sympathize with those who, like you, don't have to trade their own skin for a piece of bread.  | 80504cd8-9b21-514c-b001-4761d8c71044         |
|-----------------------|----------------------|-------------------|
| 1.In 'Harry Potter and the Philosopher's Stone', what misconception had Harry and Hermione initially had about Snape's intentions before learning the truth? <br><br>2. Hey peeps, why is Harry all jittery and pacing around the room even after telling Hermione about the whole Snape and Voldemort situation?        | Ron was asleep in the common room - apparently, he had been waiting for their return and had dozed off unnoticed. When Harry roughly shook him, Ron began to yell something about breaking the rules of a game, as if he were dreaming about a Quidditch match. However, after a few seconds, Ron completely woke up and, with his eyes wide open, listened to the story of Hermione and Harry. Harry was so excited that he could not sit still and paced back and forth across the room, trying to stay as close to the fireplace as possible. He was still shaking with cold. 'Snape wants to steal the stone for Voldemort. And Voldemort is waiting in the forest... And all this time we thought Snape wanted to steal the stone to become rich... And Voldemort...'  | 5ad25a92-28d9-5971-a81b-4f795898eeab         |
|-----------------------|----------------------|-------------------|
| 1. Hey fellow gamers, in The Hunger Games universe, if you were in a match where your ally was taken down first like Rue, how would you strategize your next move to survive against top opponents like Cato?<br><br> 2. In the 'Hunger Games' novel, why does Cato decide to spare Katniss's life after their encounter?    | What was she babbling about? You're Rue's ally? - I... I... we teamed up. We blew up the food of the Pros. I wanted to save her. Really did. But he found her first, the guy from District One - I say. Perhaps if Cato knows I helped Rue, he will kill me quickly and painlessly. - Did you kill him? - he asks grimly. - Yes. I killed him. And I covered her body with flowers. I sang to her till she fell asleep. Tears well up in my eyes. Will and strength are leaving me. There's only Rue, the pain in my head, fear of Cato and the moan of the dying girl. - Fell asleep? - mocks Cato. - Died. I sang to her till she died - I say. - Your district... sent me bread. I raise my hand - not for an arrow; I won't have time anyway. I just blow my nose. - Cato, make it quick, okay? His face shows conflicting emotions. Cato puts down the rock and says with almost a reproach: - This time, only this time, I'm letting you go. For the girl. We are even. No one owes anything to anyone anymore, understand? I nod, because I do understand. Understand about debts. About how bad it is to have them. Understand that if Cato wins, he will return to a district that has forgotten the rules to thank me. And Cato is neglecting them, too. Right now, he's not going to crack my head with a stone.  | b317200c-7fd3-5804-bbe4-bff33432ad0e         |
|-----------------------|----------------------|-------------------|

</small>

In [46]:
ds_adapter_for_index = DatasetRecordAdapter.from_source(
    dataset_name_or_path=dataset_name_or_path,
    lazy=True,
    content_col=content_col,
    queries_col=labels_col,
    chunk_id_col=chunk_id_col,
    dataframe_col=title_col,
    keywords_col="keywords_or_phrases",
    preserve_all_fields=False,
    filter_fields=[content_col, chunk_id_col],
)

In [47]:
next(ds_adapter_for_index.iterator())

{'content': "Inside was the first beautiful thing I'd seen in District 13: a replica of a meadow, filled with real trees and blooming plants, and plenty of hummingbirds fluttering about. Beetee was sitting motionless in a wheelchair in the middle of the meadow, watching a tender green bird hovering in the air and drinking nectar from a large blossom of an orange tree.",
 'content_type': 'text',
 'dataframe': 'Сойка-пересмешница',
 'keywords': None,
 'score': None,
 'meta': {'labels': ["What elements of nature were present in the artificial meadow created in District 13 in the 'Mockingjay' universe?",
   "In the 'Mockingjay' novel, who is observed sitting in a wheelchair in the middle of the recreated meadow in District 13, and what are they watching?",
   'Hey fellow tributes! 🏹 Did you notice Beetee in a wheelchair in that peaceful meadow in District 13? What do you think caused him to be in that condition? #Mockingjay',
   "What's up, book lovers? 📚 Just read about the stunning meado

### Modeling

 > See <a href="https://huggingface.co/intfloat/multilingual-e5-large">E5 large</a>, <a href="https://huggingface.co/intfloat/multilingual-e5-base">E5 base</a>, and <a href="https://huggingface.co/intfloat/multilingual-e5-small">E5 small</a>.

 > 📎 <a href="https://arxiv.org/abs/2212.03533">Paper</a>

 > ❗️For this tutorial we use `intfloat/multilingual-e5-base` as a trade-off between quality and speed.

In [27]:
model_name_or_path = "intfloat/multilingual-e5-base"

from justatom.modeling.mask import ILanguageModel
from justatom.running.encoders import EncoderRunner
from justatom.processing import RuntimeProcessor, ITokenizer

lm_model = ILanguageModel.load(model_name_or_path)

/home/justatom/miniconda3/envs/jarvis/lib/python3.12/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(
2026-02-16 06:57:12.003 | INFO     | justatom.modeling.mask:load:149 - Loading from huggingface hub via "intfloat/multilingual-e5-base"


In [28]:
def maybe_cuda_or_mps():
    if torch.backends.mps.is_built():
        return "mps"
    elif torch.cuda.is_available():
        return "cuda:0"
    else:
        return "cpu"

In [29]:
device = maybe_cuda_or_mps()

In [62]:
runner = EncoderRunner(model=lm_model, prediction_heads=[], device=device)

2026-02-16 07:16:52.840 | INFO     | justatom.running.encoders:to:36 - Moving to device cuda:0


In [31]:
processor = RuntimeProcessor(ITokenizer.from_pretrained(model_name_or_path))

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute


❗️According to the <a href="https://arxiv.org/abs/2212.03533">paper</a> E5 family is trained in an asymmetric way, meaning:

 > Use `"query: "` and `"passage: "` respectively for asymmetric tasks such as passage retrieval in open QA, ad-hoc information retrieval.

> Use `"query: "` prefix for symmetric tasks such as semantic similarity, bitext mining, paraphrase retrieval.

> Use `"query: "` prefix if you want to use embeddings as features, such as linear probing classification, clustering.

In [35]:
processor.prefix = "passage: "

> Let's put everything together in one abstraction — `Indexer`.

In [34]:
from justatom.running.indexer import API as IndexerAPI

# 1. "embedding" is the way to index the given ANN store (weaviate)
# 2. runner is responsible for mapping docs to embeddings
# 3. processor is responsible for tokenizing given chunks
# 4. device - compute everything on selected `device`

ix_runner = IndexerAPI.named("embedding", runner=runner, store=store, processor=processor, device=device)

2026-02-16 06:57:30.193 | INFO     | justatom.running.encoders:to:36 - Moving to device cuda:0


In [48]:
n_added_docs = await ix_runner.index(ds_adapter_for_index.iterator(), batch_size=32, batch_size_per_request=32)

0it [00:00, ?it/s]

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

1it [00:00,  2.58it/s]

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

2it [00:00,  3.57it/s]

...

154it [00:33,  4.21it/s]

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

155it [00:33,  4.23it/s]

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

156it [00:33,  4.62it/s]
156it [00:33,  4.62it/s]


❗️According to the <a href="https://arxiv.org/abs/2212.03533">paper</a>, E5 is trained asymmetrically, so we switch the prefix back to `query: `.

In [49]:
processor.prefix = "query: "

In [50]:
queries = [
    "thinking about 'The Hunger Games' mechanics, if you were in the same shoes as Gale, entering your name forty-two times to feed your fam, how would you strategize your game in the actual Arena? Would you team up or go solo based on these high stakes?",
    "In the universe of 'The Hunger Games', what are tesserae and what do they offer to the participants in the Harvest?",
    "In 'Harry Potter and the Philosopher's Stone', what misconception had Harry and Hermione initially had about Snape's intentions before learning the truth?",
    "Hey peeps, why is Harry all jittery and pacing around the room even after telling Hermione about the whole Snape and Voldemort situation?",
    "Hey fellow gamers, in The Hunger Games universe, if you were in a match where your ally was taken down first like Rue, how would you strategize your next move to survive against top opponents like Cato?",
    "In the 'Hunger Games' novel, why does Cato decide to spare Katniss's life after their encounter?"
] * 1

In [51]:
from justatom.running.retriever import API as RetrieverApi


def _first_document(response):
    if not isinstance(response, list) or len(response) == 0:
        return None
    first = response[0]
    if isinstance(first, list):
        return first[0] if len(first) > 0 else None
    return first

#### Pure keyword search

In [52]:
retriever = RetrieverApi.named("keywords", store=store)

In [53]:
filters = {
    "operator": "AND",
    "conditions": [
        {
            "field": "dataframe",
            "operator": "==",
            "value": "Сойка-пересмешница"
        }
    ]
}

In [42]:
await store.count_documents()

4992

In [54]:
for pos, query in enumerate(queries):
    response = await retriever.retrieve_topk(query, top_k=1, filters=filters)
    doc = _first_document(response)
    print(doc.content if doc is not None else "<EMPTY>")
    if pos < len(queries) - 1:
        print("\n")

2026-02-16 07:15:06.876 | INFO     | justatom.storing.weaviate:search_by_keywords:635 - SEARCH | algo=[BM25] | collection_name=[Justatom_collection_v2]
2026-02-16 07:15:06.887 | INFO     | justatom.storing.weaviate:search_by_keywords:635 - SEARCH | algo=[BM25] | collection_name=[Justatom_collection_v2]
2026-02-16 07:15:06.894 | INFO     | justatom.storing.weaviate:search_by_keywords:635 - SEARCH | algo=[BM25] | collection_name=[Justatom_collection_v2]
2026-02-16 07:15:06.901 | INFO     | justatom.storing.weaviate:search_by_keywords:635 - SEARCH | algo=[BM25] | collection_name=[Justatom_collection_v2]
2026-02-16 07:15:06.909 | INFO     | justatom.storing.weaviate:search_by_keywords:635 - SEARCH | algo=[BM25] | collection_name=[Justatom_collection_v2]
2026-02-16 07:15:06.919 | INFO     | justatom.storing.weaviate:search_by_keywords:635 - SEARCH | algo=[BM25] | collection_name=[Justatom_collection_v2]


Wearing headphones, I heard Gale's voice telling me to come back. However, the Hunger Games backpack reminded me of something else. Hooking the bag's strap over the back of the chair, I sprinted up the steps to my bedroom. Inside the closet, there was my father's hunting jacket. Before the Suppression, I had brought it here from our old home, thinking that its presence would calm my mother and sister when I die.


Now, after all the hustle and bustle is over and we've reached our goal, I realize I have no idea what I'm going to face in District 8. In fact, I know nothing about the state of the war or what victory will cost. Or, what will happen if we win. Plutarch tries to explain everything to me in simple words. First of all, every district is now at war with the Capitol, except for the second one, which has always been under the patronage of our enemies, despite their participation in the Hunger Games. They received more food and better living conditions. After the Dark Days and the

#### Embedding search

In [55]:
retriever = RetrieverApi.named("embedding", store=store, runner=runner, processor=processor, device=device)

In [56]:
len(queries)

6

In [57]:
for pos, query in enumerate(queries):
    response = await retriever.retrieve_topk(query, top_k=1)
    doc = _first_document(response)
    print(doc.content if doc is not None else "<EMPTY>")
    if pos < len(queries) - 1:
        print("\n")

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

And here's where the real interest begins. Suppose you're poor and starving. Then you can ask to be included in the Harvest more times than you're entitled to, and in return you'd get tesserae. They give you grain and oil for a whole year for one tessera per person. You won't be full, but it's better than nothing. You can take tesserae for the whole family. When I was twelve, my name was entered four times. Once by law, and once more for tesserae for Prim, my mother, and myself. The next years had to do the same. And since the price of a tessera increases by one entry each year, now that I've turned sixteen, my name will be on twenty cards. Gale is eighteen, and he's been feeding a family of five for seven years. His name will be entered forty two times! It's clear that people like Madge, who has never had to risk because of tesserae, annoy Gale. Next to us, the inhabitants of the slag heap, she simply has no chance of getting into the games. Well, almost no chance. Of course, the rule

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

And here's where the real interest begins. Suppose you're poor and starving. Then you can ask to be included in the Harvest more times than you're entitled to, and in return you'd get tesserae. They give you grain and oil for a whole year for one tessera per person. You won't be full, but it's better than nothing. You can take tesserae for the whole family. When I was twelve, my name was entered four times. Once by law, and once more for tesserae for Prim, my mother, and myself. The next years had to do the same. And since the price of a tessera increases by one entry each year, now that I've turned sixteen, my name will be on twenty cards. Gale is eighteen, and he's been feeding a family of five for seven years. His name will be entered forty two times! It's clear that people like Madge, who has never had to risk because of tesserae, annoy Gale. Next to us, the inhabitants of the slag heap, she simply has no chance of getting into the games. Well, almost no chance. Of course, the rule

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

...

Ron was asleep in the common room - apparently, he had been waiting for their return and had dozed off unnoticed. When Harry roughly shook him, Ron began to yell something about breaking the rules of a game, as if he were dreaming about a Quidditch match. However, after a few seconds, Ron completely woke up and, with his eyes wide open, listened to the story of Hermione and Harry. Harry was so excited that he could not sit still and paced back and forth across the room, trying to stay as close to the fireplace as possible. He was still shaking with cold. 'Snape wants to steal the stone for Voldemort. And Voldemort is waiting in the forest... And all this time we thought Snape wanted to steal the stone to become rich... And Voldemort...'




Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

What was she babbling about? You're Rue's ally? - I... I... we teamed up. We blew up the food of the Pros. I wanted to save her. Really did. But he found her first, the guy from District One - I say. Perhaps if Cato knows I helped Rue, he will kill me quickly and painlessly. - Did you kill him? - he asks grimly. - Yes. I killed him. And I covered her body with flowers. I sang to her till she fell asleep. Tears well up in my eyes. Will and strength are leaving me. There's only Rue, the pain in my head, fear of Cato and the moan of the dying girl. - Fell asleep? - mocks Cato. - Died. I sang to her till she died - I say. - Your district... sent me bread. I raise my hand - not for an arrow; I won't have time anyway. I just blow my nose. - Cato, make it quick, okay? His face shows conflicting emotions. Cato puts down the rock and says with almost a reproach: - This time, only this time, I'm letting you go. For the girl. We are even. No one owes anything to anyone anymore, understand? I nod,

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

What was she babbling about? You're Rue's ally? - I... I... we teamed up. We blew up the food of the Pros. I wanted to save her. Really did. But he found her first, the guy from District One - I say. Perhaps if Cato knows I helped Rue, he will kill me quickly and painlessly. - Did you kill him? - he asks grimly. - Yes. I killed him. And I covered her body with flowers. I sang to her till she fell asleep. Tears well up in my eyes. Will and strength are leaving me. There's only Rue, the pain in my head, fear of Cato and the moan of the dying girl. - Fell asleep? - mocks Cato. - Died. I sang to her till she died - I say. - Your district... sent me bread. I raise my hand - not for an arrow; I won't have time anyway. I just blow my nose. - Cato, make it quick, okay? His face shows conflicting emotions. Cato puts down the rock and says with almost a reproach: - This time, only this time, I'm letting you go. For the girl. We are even. No one owes anything to anyone anymore, understand? I nod,

#### Hybrid search (embeddings + keywords)
 > ❓How do we combine them? We use `alpha` in `[0.0, 1.0]`.

 > `alpha = 0.0` → pure keyword search.
 > `alpha = 1.0` → pure embedding search.

 > ⚠️ Pass `device=device` when creating the hybrid retriever to avoid implicit fallback to CPU and device mismatch errors.

In [63]:
retriever = RetrieverApi.named("hybrid", store=store, processor=processor, runner=runner, device=device)

In [64]:
for pos, query in enumerate(queries):
    response = await retriever.retrieve_topk(query, top_k=1, alpha=0.78)
    doc = _first_document(response)
    print(doc.content if doc is not None else "<EMPTY>")
    if pos < len(queries) - 1:
        print("\n")

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

And here's where the real interest begins. Suppose you're poor and starving. Then you can ask to be included in the Harvest more times than you're entitled to, and in return you'd get tesserae. They give you grain and oil for a whole year for one tessera per person. You won't be full, but it's better than nothing. You can take tesserae for the whole family. When I was twelve, my name was entered four times. Once by law, and once more for tesserae for Prim, my mother, and myself. The next years had to do the same. And since the price of a tessera increases by one entry each year, now that I've turned sixteen, my name will be on twenty cards. Gale is eighteen, and he's been feeding a family of five for seven years. His name will be entered forty two times! It's clear that people like Madge, who has never had to risk because of tesserae, annoy Gale. Next to us, the inhabitants of the slag heap, she simply has no chance of getting into the games. Well, almost no chance. Of course, the rule

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

And here's where the real interest begins. Suppose you're poor and starving. Then you can ask to be included in the Harvest more times than you're entitled to, and in return you'd get tesserae. They give you grain and oil for a whole year for one tessera per person. You won't be full, but it's better than nothing. You can take tesserae for the whole family. When I was twelve, my name was entered four times. Once by law, and once more for tesserae for Prim, my mother, and myself. The next years had to do the same. And since the price of a tessera increases by one entry each year, now that I've turned sixteen, my name will be on twenty cards. Gale is eighteen, and he's been feeding a family of five for seven years. His name will be entered forty two times! It's clear that people like Madge, who has never had to risk because of tesserae, annoy Gale. Next to us, the inhabitants of the slag heap, she simply has no chance of getting into the games. Well, almost no chance. Of course, the rule

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

...

Ron was asleep in the common room - apparently, he had been waiting for their return and had dozed off unnoticed. When Harry roughly shook him, Ron began to yell something about breaking the rules of a game, as if he were dreaming about a Quidditch match. However, after a few seconds, Ron completely woke up and, with his eyes wide open, listened to the story of Hermione and Harry. Harry was so excited that he could not sit still and paced back and forth across the room, trying to stay as close to the fireplace as possible. He was still shaking with cold. 'Snape wants to steal the stone for Voldemort. And Voldemort is waiting in the forest... And all this time we thought Snape wanted to steal the stone to become rich... And Voldemort...'




Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

What was she babbling about? You're Rue's ally? - I... I... we teamed up. We blew up the food of the Pros. I wanted to save her. Really did. But he found her first, the guy from District One - I say. Perhaps if Cato knows I helped Rue, he will kill me quickly and painlessly. - Did you kill him? - he asks grimly. - Yes. I killed him. And I covered her body with flowers. I sang to her till she fell asleep. Tears well up in my eyes. Will and strength are leaving me. There's only Rue, the pain in my head, fear of Cato and the moan of the dying girl. - Fell asleep? - mocks Cato. - Died. I sang to her till she died - I say. - Your district... sent me bread. I raise my hand - not for an arrow; I won't have time anyway. I just blow my nose. - Cato, make it quick, okay? His face shows conflicting emotions. Cato puts down the rock and says with almost a reproach: - This time, only this time, I'm letting you go. For the girl. We are even. No one owes anything to anyone anymore, understand? I nod,

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

What was she babbling about? You're Rue's ally? - I... I... we teamed up. We blew up the food of the Pros. I wanted to save her. Really did. But he found her first, the guy from District One - I say. Perhaps if Cato knows I helped Rue, he will kill me quickly and painlessly. - Did you kill him? - he asks grimly. - Yes. I killed him. And I covered her body with flowers. I sang to her till she fell asleep. Tears well up in my eyes. Will and strength are leaving me. There's only Rue, the pain in my head, fear of Cato and the moan of the dying girl. - Fell asleep? - mocks Cato. - Died. I sang to her till she died - I say. - Your district... sent me bread. I raise my hand - not for an arrow; I won't have time anyway. I just blow my nose. - Cato, make it quick, okay? His face shows conflicting emotions. Cato puts down the rock and says with almost a reproach: - This time, only this time, I'm letting you go. For the girl. We are even. No one owes anything to anyone anymore, understand? I nod,